In [19]:
import numpy as np
import scipy.spatial as spa
import scipy 
import prody
from pathlib import Path
import importlib
from typing import Union, List, Dict

from data_processing.pdbbind_dataset import rec_atom_elements

import rdkit

import dgl
import torch

%matplotlib widget
from matplotlib import pyplot as plt

# define hard-coded filepaths

In [29]:
DEV_INDEX_PATH = '/home/ian/projects/mol_diffusion/ligdiff/data/PDBbind_processed/train_index_mini.txt'
DEV_DATA_DIR = '/home/ian/projects/mol_diffusion/ligdiff/data/PDBbind/refined-set'
DEV_PROCESSED_DATA_DIR = '/home/ian/projects/mol_diffusion/ligdiff/data/PDBbind_processed'

DEV_DATA_DIR = Path(DEV_DATA_DIR)

# import code for processing pdbbind

In [40]:
try:
    importlib.reload(process_pdbbind)
except NameError:
    import data_processing.pdbbind_processing as process_pdbbind

parse_protein = process_pdbbind.parse_protein
parse_ligand = process_pdbbind.parse_ligand
get_pocket_atoms = process_pdbbind.get_pocket_atoms

# define dataset class for PDB

In [41]:
class PDBbind(dgl.data.DGLDataset):

    def __init__(self, name: str, index_fpath: str, 
        raw_data_dir: str,
        processed_data_dir: str,
        rec_elements: List[str],
        lig_box_padding: Union[int, float] = 6,
        pocket_cutoff: Union[int, float] = 4,
        receptor_k: int = 3,
        dataset_size: int = None):
        
        self.dataset_size: int = dataset_size

        # define filepaths of data
        self.index_fpath: Path = Path(index_fpath)
        self.raw_data_dir: Path = Path(raw_data_dir)
        self.processed_data_dir: Path = Path(processed_data_dir)

        # atom typing configurations
        self.rec_elements = rec_elements
        self.rec_element_map: Dict[str, int] = { element: idx for idx, element in enumerate(self.rec_elements) }
        self.rec_element_map['other'] = len(self.rec_elements)


        # hyperparameters for protein graph
        self.receptor_k: int = receptor_k
        self.lig_box_padding: Union[int, float] = lig_box_padding
        self.pocket_cutoff: Union[int, float] = pocket_cutoff



        super().__init__(name=name) # this has to happen last because this will call self.process()

    def __getitem__(self, i):
        pass

    def __len__(self):
        return 1

    def process(self):

        # TODO: implement logic that only goes makes the graph objects if they don't already exist

        # get pdb ids from index file
        with open(self.index_fpath, 'r') as f:
            pdb_ids = [line.strip() for line in f]

        if self.dataset_size is not None:
            pdb_ids = pdb_ids[:self.dataset_size]

        # we will want to do this paralellized over PDBs
        # but for now, a simple for loop will do
        for pdb_id in pdb_ids:

            # get all atoms from pdb file
            pdb_atoms: prody.AtomGroup = parse_protein(pdb_id, data_dir=self.raw_data_dir)

            # get rdkit molecule from ligand, as well as atom types and charges
            ligand, lig_atom_positions, lig_atom_features = parse_ligand(pdb_id, data_dir=self.raw_data_dir)

            # get all protein atoms that form the binding pocket
            pocket_atom_positions, pocket_atom_features \
                 = get_pocket_atoms(pdb_atoms, lig_atom_positions, box_padding=self.lig_box_padding, pocket_cutoff=self.pocket_cutoff, element_map=self.rec_element_map)
        


# method for finding binding pocket atoms

In [42]:
dataset = PDBbind(name='dev_dataset', 
                index_fpath=DEV_INDEX_PATH, 
                raw_data_dir=DEV_DATA_DIR, 
                processed_data_dir=DEV_PROCESSED_DATA_DIR, 
                rec_elements=rec_atom_elements,
                lig_box_padding=6, pocket_cutoff=4,
                receptor_k=3,
                dataset_size=1)

@> 4858 atoms and 1 coordinate set(s) were parsed in 0.10s.


In [27]:
test_pdb_id = '1n4k'
protein_atoms = parse_protein(test_pdb_id, data_dir=DEV_DATA_DIR)
ligand, lig_atom_positions, lig_atom_features = parse_ligand(test_pdb_id, data_dir=DEV_DATA_DIR)

pocket_atom_positions, pocket_atom_features = get_pocket_atoms(protein_atoms, lig_atom_positions, box_padding=6, pocket_cutoff=4, supported_elements=rec_atom_elements)

@> 4858 atoms and 1 coordinate set(s) were parsed in 0.09s.


TypeError: get_pocket_atoms() got an unexpected keyword argument 'supported_elements'

# build binding pocket graph

1. one-hot encode atom types
2. determine edges for KNN graph

In [17]:
def build_pocket_graph(atom_positions, atom_features, k=3, edge_algorithm='bruteforce'):
    # atom_positions is an array of shape (N, 3) where N is the number of atoms
    # atom_features is an array of shape (N, d) where N is the number of atoms and d is the size of the feature vector for each atom


    # construct KNN graph
    g = dgl.knn_graph(atom_positions, k=k, algorithm=edge_algorithm, dist='euclidean')
    return g

test_pdb_id = '1n4k'
protein_atoms = parse_protein(test_pdb_id, )
ligand, lig_atom_positions, lig_atom_types, lig_atom_charges = parse_ligand(test_pdb_id)
pocket_atom_positions, pocket_atom_types, pocket_atom_charges = get_pocket_atoms(protein_atoms, lig_atom_positions, cutoff=6)

g = build_pocket_graph(pocket_atom_positions, None)

TypeError: parse_protein() missing 1 required positional argument: 'data_dir'